In [1]:
import pandas as pd
import cufflinks as cf
import warnings
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chisquare
from  functools import reduce
import re
from plotly.offline import plot,iplot
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.express as px#graficos express
import sys
warnings.filterwarnings("ignore")
cf.go_offline()
pd.set_option("display.max_columns",200)

In [2]:
def chi_square(df,col,valor_miss):
    x_i=df[col].fillna(valor_miss).value_counts()
    k=x_i.sum()
    p_i=df[col].dropna().value_counts(1)
    m_i=k*p_i
    chi=chisquare(f_obs=x_i,f_exp=m_i)
    p_val=chi.pvalue
    alpha=0.05
    if p_val<alpha:
        print("Rechazamos HO(La porporción de categorias es la misma que la general)")
    else:
        print("Aceptamos HO(La porporción de categorias es la misma que la general)")

In [3]:
def other(df,col,n):
    aux=list(df[col].value_counts(1).index[n:].values)
    dictio_aux=dict(zip(aux,["otro"]*len(aux)))
    df[col]=df[col].replace(dictio_aux)
    return df

def multiple_replace(string, rep_dict):
    pattern = re.compile("|".join([re.escape(k) for k in sorted(rep_dict,key=len,reverse=True)]), flags=re.DOTALL)
    return pattern.sub(lambda x: rep_dict[x.group(0)]+"_", string)

In [4]:
## Multicolinealidad Categoricas##
def multicolinealidad_cate(df,y,threshold=5,only_final_vif=True):
    '''
    Función para calcular el VIF Genralizado, (GVIF, Fox and Monette 1992)
    Las variables no deben de deben de estar transformadas con OneHotEncoder ya que la función lo realiza internamnete.
    Args:
        df
        y
        threshold
        only_final_vif
    
    Returns:
        pandas data frame: a data frame, indexed by factor of the GVIF, GVIF^(1/2Df), VIF^(1/2Df)^2 
        dictionary: Dictionary of column names (keys) and GVIF ** (1 / (2 * Df)) ** 2 (values)
    '''

    df_x = df.drop(y,axis=1)
    #Guardamos los nombres de las columnas añadiendo el prefijo

    onehot_list = list(df_x.select_dtypes(include=['category', 'object', 'string']).columns)
    #Generamos las variables dummy 
    
    df_1hot = pd.get_dummies(df_x, drop_first=True, dummy_na=False, prefix_sep='_')


    #Dataframe vacio para guardar los resultados
    gvif_df = pd.DataFrame(columns = ['factor','GVIF','Df','GVIF^(1/2Df)', 'VIF'])

    # Iteramos sobre las columas
    for (columnName, columnData) in df_x.iteritems():
        #Si se generaron las de una columna, es decir la variable tiene más de dos posibilidades utilizamos todas las variables creadas
        
        if columnName in onehot_list:
            X1 = df_1hot.loc[:, df_1hot.columns.str.startswith(columnName)]
            X2 = df_1hot.loc[:, ~df_1hot.columns.str.startswith(columnName)]
        else:
            X1 = df_1hot[[columnName]].values
            X2 = df_1hot.loc[:, df_1hot.columns != columnName].values
        display(X1)
        # Calculamose gvif
        #A la matriz de correlación de las variables codificadas en caliente del atributo en consideración.
        #B la matriz de correlación de todos los demás atributos del conjunto de datos (uno codificado en dummy y numérico) excluyendo los de A.
        #C la matriz de correlación de las variables consideradas tanto en A como en B.
        #GVIF= (det(A)*det(B))/det(C)
        display(np.corrcoef(X1, rowvar=False))
        gvif = np.linalg.det(np.array(np.corrcoef(X1, rowvar=False), ndmin=2)) * np.linalg.det(np.corrcoef(X2, rowvar=False)) / np.linalg.det(np.corrcoef(np.append(X1, X2, axis=1), rowvar=False))
        #(GVIF) elevado a (1 / (2 * grados de libertad))
        #Los grados de libertad es 1-n (n=Número de opciones que tiene la variable categorica)
        gvif_12df = np.power(gvif, 1 / (2 * X1.shape[1]))
        gvif_12df_sq = gvif_12df ** 2
        DF =  X1.shape[1]
    

        # 
        new_row = {'factor': columnName, 'GVIF': gvif,'Df':DF, 'GVIF^(1/2Df)': gvif_12df, 'VIF': gvif_12df_sq}
        gvif_df = gvif_df.append(new_row, ignore_index=True)

    gvif_df = gvif_df.set_index('factor')
    if only_final_vif:
        gvif_df_final = gvif_df.drop(['GVIF','Df','GVIF^(1/2Df)'],axis=1)
    else:
        gvif_df_final = gvif_df 
    gvif_filter = gvif_df.loc[gvif_df['VIF'] >= threshold]['VIF'].to_dict()
    if gvif_filter:
        for i in gvif_filter.keys():
            df_x_m = df_x.drop([i],axis=1)
    else:
        df_x_m = df_x
    df_m=pd.concat([df_x_m,df[y]],axis=1)
    return gvif_df_final,gvif_filter,df_m


In [5]:
sys.path.append("../libreria/")

In [6]:
from libreria import rename_cols,completitud,text_clean,box,histogram

In [7]:
df=pd.read_csv("../data/visualizaciones/dataset_house_prices.csv")

## Variables

**El problema**

En análisis exploratorio tiene el objetivo de entender el significado y la relevancia de cada variable con respecto al problema. Además apartir de este análisis se pueden determinar que acciones tomar para cada variable En este caso:

- La relevancia de la variable en la compra de una casa.
- La importancia de la variable.
- Solapamiento con otras variables.


Variable objetivo : 'SalePrice'.

- MSSubClass: clase de construcción
- MSZoning: clasificación de la zona
- LotFrontage: pies lineales de calle de la parcela
- LotArea: tamaño de la parcela en pies cuadrados
- Street: tipo de acceso por carretera
- Alley: tipo de acceso al callejón
- LotShape: forma de la parcela
- LandContour: planitud de la parcela
- Utilities: servicios públicos disponibles
- LotConfig: Configuración de parcela
- LandSlope: pendiente de la parcela
- Neighborhood: ubicación física dentro de los límites de la ciudad de Ames
- Condition1: proximidad a la carretera principal o al ferrocarril
- Condition2: proximidad a la carretera principal o al ferrocarril (si hay un segundo)
- BldgType: tipo de vivienda
- HouseStyle: estilo de vivienda
- OverallQual: calidad general del material y del acabado
- OverallCond: condición general
- YearBuilt: fecha original de construcción
- YearRemodAdd: fecha de remodelación
- RoofStyle: tipo de cubierta
- RoofMatl: material del techo
- Exterior1st: revestimiento exterior de la casa
- Exterior2nd: revestimiento exterior de la casa (si hay más de un material)
- MasVnrType: tipo de revestimiento de mampostería
- MasVnrArea: área de revestimiento de mampostería en pies cuadrados
- ExterQual: calidad del material exterior
- ExterCond: estado del material en el exterior
- Foundation: tipo de cimentación
- BsmtQual: altura del sótano
- BsmtCond: estado general del sótano
- BsmtExposure: paredes del sótano a nivel de calle o de jardín
- BsmtFinType1: calidad del área acabada del sótano
- BsmtFinSF1: pies cuadrados de la superficie acabada tipo 1
- BsmtFinType2: calidad de la segunda superficie acabada (si existe)
- BsmtFinSF2: Pies cuadrados de la superficie acabada tipo 2
- BsmtUnfSF: pies cuadrados del área sin terminar del sótano
- TotalBsmtSF: pies cuadrados totales del sótano
- Heating: tipo de calefacción
- HeatingQC: calidad y estado de la calefacción
- CentralAir: aire acondicionado central
- Electrical: sistema eléctrico
- 1erFlrSF: área en pies cuadrados de la primera planta (o planta baja)
- 2ndFlrSF: área en pies cuadrados de la segunda planta
- LowQualFinSF: pies cuadrados acabados de baja calidad (todos los pisos)
- GrLivArea: superficie habitable por encima del nivel del suelo en pies cuadrados
- BsmtFullBath: cuartos de baño completos en el sótano
- BsmtHalfBath: medio baño del sótano
- FullBath: baños completos sobre el nivel del suelo
- HalfBath: medios baños sobre el nivel del suelo
- Bedroom: número de dormitorios por encima del nivel del sótano
- Kitchen: número de cocinas
- KitchenQual: calidad de la cocina
- TotRmsAbvGrd: total de habitaciones por encima del nivel del suelo (no incluye baños)
- Functional: valoración de la funcionalidad de la vivienda
- Fireplaces: número de chimeneas
- FireplaceQu: calidad de la chimenea
- GarageType: ubicación del garaje
- GarageYrBlt: año de construcción del garaje
- GarageFinish: acabado interior del garaje
- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados
- GarageQual: calidad de garaje
- GarageCond: condición de garaje
- PavedDrive: calzada asfaltada
- WoodDeckSF: area de plataforma de madera en pies cuadrados
- OpenPorchSF: área de porche abierto en pies cuadrados
- EnclosedPorch: área de porche cerrada en pies cuadrados
- 3SsnPorch: área de porche de tres estaciones en pies cuadrados
- ScreenPorch: superficie acristalada del porche en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados
- PoolQC: calidad de la piscina
- Fence: calidad de la valla
- MiscFeature: característica miscelánea no cubierta en otras categorías
- MiscVal: valor en dólares de la característica miscelánea
- MoSold: mes de venta
- YrSold: año de venta
- SaleType: tipo de venta
- SaleCondition: Condiciones de venta

In [8]:
v_feats=["MSSubClass","MSZoning","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope","Condition1","Condition2","BldgType","HouseStyle","OverallQual","OverallCond",
        "YearBuilt","YearRemodAdd","RoofStyle","RoofMatl","MasVnrType","ExterQual",
        "ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
        "Heating","HeatingQC","CentralAir","Electrical","LowQualFinSF","BsmtHalfBath","FullBath","KitchenAbvGr","KitchenQual","Functional","Fireplaces","FireplaceQu",
        "GarageType","GarageYrBlt","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence",
        "MiscFeature","MoSold","YrSold","SaleType","SaleCondition"]
c_feats=["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","1stFlrSF",
        "2ndFlrSF","GrLivArea","GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch",
        "PoolArea","MiscVal","BedroomAbvGr","HalfBath","GarageCars","BsmtFullBath","TotRmsAbvGrd"]
tgt=["SalePrice"]

df=rename_cols(df,v_feats,"v_")
df=rename_cols(df,c_feats,"c_")
df=rename_cols(df,tgt,"tgt_")

In [9]:
#Eliminamos aquellas variables que que hacen referencia a la venta
df.drop(columns=["v_SaleType","v_SaleCondition"],inplace=True)

In [10]:
df.drop(columns=["Neighborhood","Exterior2nd","Exterior1st","v_YrSold","v_MoSold"],inplace=True)

In [11]:
#NO hay duplicados
df[df.duplicated(subset=["Id"])]

,Id,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_Alley,v_LotShape,v_LandContour,v_Utilities,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,c_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_FireplaceQu,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,v_PoolQC,v_Fence,v_MiscFeature,c_MiscVal,tgt_SalePrice


In [12]:
#Variables para limpiar el texto
features=list(filter(lambda x: x not in ["v_GarageYrBlt","v_YearBuilt"],df.filter(like="v_") ))


for col in features:
    df[col]=df[col].fillna("nan").astype(str).map(text_clean).replace("nan",np.nan)

In [13]:
histogram(df,"tgt_SalePrice",5,"tgt_SalePrice").show()

In [14]:
miss=completitud(df)

In [15]:
miss

,columna,total,completitud
0,v_PoolQC,1453,0.479452
1,v_MiscFeature,1406,3.698630
2,v_Alley,1369,6.232877
3,v_Fence,1179,19.246575
4,v_FireplaceQu,690,52.739726
...,...,...,...
69,c_MiscVal,0,100.000000
70,v_HeatingQC,0,100.000000
71,v_CentralAir,0,100.000000
72,c_2ndFlrSF,0,100.000000


In [16]:
null_cols=list(miss[miss["completitud"]<80]["columna"].values)

In [17]:
null_cols

['v_PoolQC', 'v_MiscFeature', 'v_Alley', 'v_Fence', 'v_FireplaceQu']

In [18]:
df=df.drop(columns=null_cols)

In [19]:
df["v_MSSubClass"]=df["v_MSSubClass"].astype(int)

In [20]:
df_multi=df.copy()

In [21]:
df_multi=df.copy()

## Isolation Forest

**La clase sklearn.ensemble.IsolationForest incorpora las principales funcionalidades que se necesitan a la hora de trabajar con modelos Isolation Forest. Los principales argumentos para entrenar este tipo de modelos son:**

- n_estimators: número de árboles que forman el modelo.

- max_samples: número de observaciones empleadas para entrenar cada árbol.

- max_features : El número de características que se extraerán de X para entrenar cada estimador base.

- contamination: proporción de anomalías esperadas en los datos de entrenamiento. En base a este valor, se establece el límite acorde al cual se clasifican las observaciones en normales o anómalas.

- random_state: semilla para garantizar la reproducibilidad de los resultados.

- Se procede a entrenar un modelo asumiendo que hay un 1% de observaciones anómalas en el conjunto de entrenamiento.

In [22]:
from sklearn.ensemble import IsolationForest

In [23]:
df=df.drop(columns=["Id"])

In [24]:
df.shape

(1460, 68)

In [25]:
df_iso=df.filter(like="c_").dropna()

In [26]:
#max_features=1
max_features=df_iso.shape[1]
n_estimators=50
max_samples='auto'
contamination=float(0.2)
model=IsolationForest(max_features = max_features, n_estimators=n_estimators, max_samples=max_samples, contamination=contamination)
model.fit(df_iso)

IsolationForest(contamination=0.2, max_features=23, n_estimators=50)

- Los modelos IsolationForest tienen dos métodos de predicción con los que se obtiene distinta información.

- Con el método predict() se devuelve directamente la clasificación de anomalía (-1) o no anomalía (1) acorde a la proporción de contaminación que se ha indicado en la definición del modelo.

In [27]:
df_iso["outlier"]=model.predict(df_iso)

In [28]:
df_iso["outlier"].value_counts()

 1    956
-1    239
Name: outlier, dtype: int64

In [29]:
df_iso[df_iso["outlier"]==-1].shape[0]/df_iso.shape[0]

0.2

In [30]:
indices=list(df_iso[df_iso["outlier"]==1].index)

In [31]:
df=df.iloc[indices]

In [32]:
df.reset_index(drop=True,inplace=True)

## MISSINGS

### Variables unitarias

In [33]:
#Eliminamos Variables unarias

In [34]:
df[["v_Utilities","v_Street"]].describe()

,v_Utilities,v_Street
count,956,956
unique,1,2
top,allpub,pave
freq,956,953


In [35]:
df.drop(columns=["v_Utilities"],inplace=True)

In [36]:
miss=completitud(df)

miss_cols=list(miss[miss["completitud"]<100]["columna"].values)

In [37]:
miss

,columna,total,completitud
0,v_GarageQual,41,95.711297
1,v_GarageType,41,95.711297
2,v_GarageYrBlt,41,95.711297
3,v_GarageFinish,41,95.711297
4,v_GarageCond,41,95.711297
...,...,...,...
62,c_BsmtUnfSF,0,100.000000
63,c_TotalBsmtSF,0,100.000000
64,c_MiscVal,0,100.000000
65,v_YearBuilt,0,100.000000


In [38]:
miss_cols

['v_GarageQual',
 'v_GarageType',
 'v_GarageYrBlt',
 'v_GarageFinish',
 'v_GarageCond',
 'v_BsmtExposure',
 'v_BsmtFinType2',
 'v_BsmtQual',
 'v_BsmtFinType1',
 'v_BsmtCond',
 'v_Electrical']

### NORMALIZAMOS DISCRETAS

In [39]:
v_=list(filter(lambda x:"v_" in x,miss_cols))
v_=[x for x in v_ if x!="v_GarageYrBlt"]

In [40]:
for col in v_:
    print(col)
    print(df[col].isnull().sum())
    df[col]=df[col].fillna("nan").astype(str).map(text_clean).replace("nan",np.nan)
    print(df[col].isnull().sum())

v_GarageQual
41
41
v_GarageType
41
41
v_GarageFinish
41
41
v_GarageCond
41
41
v_BsmtExposure
24
24
v_BsmtFinType2
23
23
v_BsmtQual
23
23
v_BsmtFinType1
23
23
v_BsmtCond
23
23
v_Electrical
1
1


In [41]:
for col in v_:
    print(col)
    display(df[col].value_counts(1))
    print("\n")

v_GarageQual


ta    0.938798
fa    0.044809
gd    0.010929
po    0.003279
ex    0.002186
Name: v_GarageQual, dtype: float64



v_GarageType


attchd     0.600000
detchd     0.334426
builtin    0.041530
basment    0.014208
carport    0.006557
2types     0.003279
Name: v_GarageType, dtype: float64



v_GarageFinish


...

unf    0.338692
glq    0.258307
alq    0.150054
rec    0.101822
blq    0.101822
lwq    0.049303
Name: v_BsmtFinType1, dtype: float64



v_BsmtCond


ta    0.918542
gd    0.043944
fa    0.036442
po    0.001072
Name: v_BsmtCond, dtype: float64



v_Electrical


sbrkr    0.899476
fusea    0.074346
fusef    0.021990
fusep    0.003141
mix      0.001047
Name: v_Electrical, dtype: float64

In [42]:
##"v_GarageType"
##"v_Electrical"
#MasVnrType

In [43]:
dictionary={"v_BldgType":1,"v_Foundation":2,"v_Functional":1,"v_Heating":1,"v_HouseStyle":3,"v_LandContour":1,
"v_LandSlope":1,"v_LotConfig":2,"v_LotShape":1,"v_MSZoning":2,"v_RoofMatl":1,"v_RoofStyle":2,"v_Condition2":1,"v_Condition1":1,"v_BsmtFinType2":1,"v_BsmtFinType1":3,"v_BsmtExposure":1}

In [44]:
for i,j in dictionary.items():
    print(i)
    df=other(df,i,j)
    print(df[i].value_counts())

v_BldgType
1fam    799
otro    157
Name: v_BldgType, dtype: int64
v_Foundation
pconc     415
cblock    403
otro      138
Name: v_Foundation, dtype: int64
v_Functional
typ     895
otro     61
Name: v_Functional, dtype: int64
v_Heating
gasa    934
otro     22
Name: v_Heating, dtype: int64
v_HouseStyle
1story    484
2story    276
1 5fin    121
otro       75
Name: v_HouseStyle, dtype: int64
v_LandContour
lvl     869
otro     87
Name: v_LandContour, dtype: int64
v_LandSlope
gtl     920
otro     36
Name: v_LandSlope, dtype: int64
v_LotConfig
inside    746
corner    148
otro       62
Name: v_LotConfig, dtype: int64
v_LotShape
reg     709
otro    247
Name: v_LotShape, dtype: int64
v_MSZoning
rl      718
rm      172
otro     66
Name: v_MSZoning, dtype: int64
v_RoofMatl
compshg    950
otro         6
Name: v_RoofMatl, dtype: int64
v_RoofStyle
gable    784
hip      155
otro      17
Name: v_RoofStyle, dtype: int64
v_Condition2
norm    946
otro     10
Name: v_Condition2, dtype: int64
v_Condition1
no

In [45]:
dictio_GarageType=dict(zip(list(df["v_GarageType"].value_counts().index[-3:]),["otro"]*3))
dictio_electrical=dict(zip(list(df["v_Electrical"].value_counts().index[-3:]),["otro"]*3))
dictio_MasVnrType=dict(zip(list(df["v_MasVnrType"].value_counts().index[-2:]),["otro"]*2))

In [46]:
df["v_GarageType"]=df["v_GarageType"].replace(dictio_GarageType)
df["v_Electrical"]=df["v_Electrical"].replace(dictio_electrical)
df["v_MasVnrType"]=df["v_MasVnrType"].replace(dictio_MasVnrType)

In [47]:
df["v_GarageType"].value_counts(1)

attchd     0.600000
detchd     0.334426
builtin    0.041530
otro       0.024044
Name: v_GarageType, dtype: float64

In [48]:
df["v_Electrical"].value_counts(1)

sbrkr    0.899476
fusea    0.074346
otro     0.026178
Name: v_Electrical, dtype: float64

In [49]:
## "v_GarageQual","v_GarageCond"

In [50]:
df["v_GarageQual"].value_counts(1)

ta    0.938798
fa    0.044809
gd    0.010929
po    0.003279
ex    0.002186
Name: v_GarageQual, dtype: float64

In [51]:
df["v_GarageCond"].value_counts(1)

ta    0.953005
fa    0.034973
po    0.005464
gd    0.004372
ex    0.002186
Name: v_GarageCond, dtype: float64

In [52]:
dictio_qual_cond=dict(zip(["ex","gd","ta","fa","po"],[3,3,2,1,1]))

In [53]:
feats_replace_cond_qual=["v_GarageQual","v_GarageCond","v_BsmtCond","v_BsmtQual","v_ExterCond","v_ExterQual","v_HeatingQC","v_KitchenQual"]

In [54]:
for feat in feats_replace_cond_qual:
    print(feat)
    df[feat]=df[feat].replace(dictio_qual_cond)
    print(df[feat].value_counts(1))
    print("\n")

v_GarageQual
2.0    0.938798
1.0    0.048087
3.0    0.013115
Name: v_GarageQual, dtype: float64


v_GarageCond
2.0    0.953005
1.0    0.040437
3.0    0.006557
Name: v_GarageCond, dtype: float64


v_BsmtCond
2.0    0.918542
3.0    0.043944
1.0    0.037513
Name: v_BsmtCond, dtype: float64


v_BsmtQual
2.0    0.487674
3.0    0.479100
1.0    0.033226
Name: v_BsmtQual, dtype: float64


v_ExterCond
2    0.879707
3    0.097280
1    0.023013
Name: v_ExterCond, dtype: float64


v_ExterQual
2    0.655858
3    0.335774
1    0.008368
Name: v_ExterQual, dtype: float64


v_HeatingQC
3    0.673640
2    0.291841
1    0.034519
Name: v_HeatingQC, dtype: float64


v_KitchenQual
2    0.533473
3    0.437238
1    0.029289
Name: v_KitchenQual, dtype: float64




In [55]:
miss=completitud(df)
miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,41,95.711297
1,v_GarageType,41,95.711297
2,v_GarageYrBlt,41,95.711297
3,v_GarageFinish,41,95.711297
4,v_GarageCond,41,95.711297
5,v_BsmtExposure,24,97.489540
6,v_BsmtFinType2,23,97.594142
7,v_BsmtQual,23,97.594142
8,v_BsmtFinType1,23,97.594142
9,v_BsmtCond,23,97.594142


In [56]:
#Remplazamos los valores de la fecha de construccion del garage por la fecha de construcción de la propiedad
df['v_GarageYrBlt']=df['v_GarageYrBlt'].fillna(df['v_YearBuilt']).astype(float)

In [57]:
miss=completitud(df)

miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,41,95.711297
1,v_GarageType,41,95.711297
2,v_GarageFinish,41,95.711297
3,v_GarageCond,41,95.711297
4,v_BsmtExposure,24,97.489540
5,v_BsmtQual,23,97.594142
6,v_BsmtFinType2,23,97.594142
7,v_BsmtCond,23,97.594142
8,v_BsmtFinType1,23,97.594142
9,v_Electrical,1,99.895397


### CATEGORICAS

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train,X_test=train_test_split(df,test_size=.2,random_state=413)

In [60]:
miss=completitud(df)

miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,41,95.711297
1,v_GarageType,41,95.711297
2,v_GarageFinish,41,95.711297
3,v_GarageCond,41,95.711297
4,v_BsmtExposure,24,97.489540
5,v_BsmtQual,23,97.594142
6,v_BsmtFinType2,23,97.594142
7,v_BsmtCond,23,97.594142
8,v_BsmtFinType1,23,97.594142
9,v_Electrical,1,99.895397


In [61]:
v_feats=list(filter(lambda x:"v_" in x,list(miss[miss["completitud"]<100]["columna"].values)))

In [62]:
v_feats

['v_GarageQual',
 'v_GarageType',
 'v_GarageFinish',
 'v_GarageCond',
 'v_BsmtExposure',
 'v_BsmtQual',
 'v_BsmtFinType2',
 'v_BsmtCond',
 'v_BsmtFinType1',
 'v_Electrical']

In [63]:
for column in v_feats:
    print(column)
    value=X_train[column].value_counts().index[0]
    print(value)
    print(chi_square(df,column,value))
    print("\n")

v_GarageQual
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageType
attchd
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageFinish
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageCond
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtExposure
no
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtQual
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtFinType2
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtCond
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtFinType1
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_Electrical
sbrkr
Aceptamos HO(La porporción de categorias es la misma que la general)
None




In [64]:
from sklearn.impute import SimpleImputer

In [65]:
imp=SimpleImputer(missing_values=np.nan,strategy="most_frequent")

In [66]:
imp.fit(X_train[v_feats])

SimpleImputer(strategy='most_frequent')

In [67]:
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

In [68]:
X_train[v_feats]=pd.DataFrame(imp.transform(X_train[v_feats]),columns=v_feats)

In [69]:
X_test[v_feats]=pd.DataFrame(imp.transform(X_test[v_feats]),columns=v_feats)

In [70]:
miss=completitud(X_train)

miss[miss["completitud"]<100]

,columna,total,completitud


In [71]:
miss=completitud(X_test)
miss[miss["completitud"]<100]

,columna,total,completitud


In [72]:
X_train

,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,c_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal,tgt_SalePrice
0,20,rl,66.0,7742,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,7,1966,1966,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,no,otro,763,unf,0,192,955,gasa,3,y,sbrkr,955,0,0,955,1,0,1,0,3,1,2,6,typ,0,attchd,1966.0,unf,1,386,2.0,2.0,y,0,0,0,0,0,0,0,127000
1,190,rm,33.0,4456,pave,reg,lvl,inside,gtl,norm,norm,otro,2story,4,5,1920,2008,gable,compshg,none,0.0,2,2,otro,2.0,2.0,no,unf,0,unf,0,736,736,gasa,3,y,sbrkr,736,716,0,1452,0,0,2,0,2,3,2,8,typ,0,attchd,1920.0,unf,0,0,2.0,2.0,n,0,0,102,0,0,0,0,113000
2,20,rl,60.0,9600,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,7,1950,1995,gable,compshg,none,0.0,2,2,cblock,3.0,2.0,no,otro,442,unf,0,625,1067,gasa,2,y,sbrkr,1067,0,0,1067,0,0,2,0,2,1,3,4,otro,0,attchd,1996.0,unf,2,436,2.0,2.0,y,290,0,0,0,0,0,0,128000
3,60,rl,64.0,8633,pave,reg,lvl,otro,gtl,norm,norm,1fam,2story,6,5,2005,2005,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,no,glq,193,unf,0,545,738,gasa,3,y,sbrkr,738,738,0,1476,1,0,2,1,3,1,3,7,typ,0,attchd,2005.0,fin,2,540,2.0,2.0,y,100,35,0,0,0,0,0,173500
4,30,rl,60.0,5400,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,5,6,1920,1950,gable,compshg,none,0.0,2,2,otro,1.0,2.0,no,unf,0,unf,0,691,691,gasa,3,y,fusea,691,0,0,691,0,0,1,0,2,1,3,4,typ,0,detchd,1920.0,unf,1,216,1.0,2.0,n,0,20,94,0,0,0,0,86000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,70,rl,55.0,10592,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,6,7,1923,1996,hip,compshg,none,0.0,2,3,pconc,2.0,1.0,no,unf,0,unf,0,602,602,gasa,2,y,sbrkr,900,602,0,1502,0,0,1,1,3,1,3,7,typ,2,detchd,1923.0,unf,1,180,2.0,2.0,y,96,0,112,0,53,0,0,165000
760,60,rl,74.0,8834,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,9,5,2004,2005,hip,compshg,otro,216.0,3,2,pconc,3.0,2.0,no,glq,1170,unf,0,292,1462,gasa,3,y,sbrkr,1462,762,0,2224,1,0,2,1,4,1,3,10,typ,1,attchd,2004.0,fin,3,738,2.0,2.0,y,184,0,0,0,0,0,0,350000
761,20,rl,85.0,10182,pave,otro,lvl,otro,gtl,otro,norm,1fam,1story,8,5,2006,2006,hip,compshg,otro,420.0,3,2,pconc,3.0,2.0,otro,glq,1220,unf,0,440,1660,gasa,3,y,sbrkr,1660,0,0,1660,1,0,2,0,3,1,3,8,typ,1,attchd,2006.0,rfn,2,500,2.0,2.0,y,322,50,0,0,0,0,0,290000
762,50,rl,60.0,9738,pave,reg,lvl,inside,gtl,norm,norm,1fam,1 5fin,5,7,1924,1950,gable,compshg,none,0.0,2,3,otro,2.0,2.0,no,otro,392,unf,0,392,784,gasa,3,y,sbrkr,949,272,0,1221,1,0,1,0,4,1,2,7,typ,0,attchd,1965.0,unf,1,392,2.0,2.0,y,0,0,236,0,0,0,0,104900


In [73]:
X_test

,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,c_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal,tgt_SalePrice
0,80,rl,80.0,8816,pave,reg,lvl,inside,gtl,norm,norm,1fam,otro,6,7,1971,1971,gable,compshg,brkface,80.0,2,2,cblock,2.0,2.0,otro,glq,504,unf,0,506,1010,gasa,3,y,sbrkr,1052,0,0,1052,1,0,1,0,3,1,2,6,typ,0,attchd,1971.0,unf,2,440,2.0,2.0,y,0,0,0,0,0,0,0,155000
1,20,rl,73.0,8899,pave,otro,lvl,inside,gtl,norm,norm,1fam,1story,7,5,2007,2007,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,otro,glq,24,unf,0,1316,1340,gasa,3,y,sbrkr,1340,0,0,1340,0,0,2,0,3,1,3,6,typ,0,attchd,2007.0,fin,2,396,2.0,2.0,y,100,30,0,0,0,0,0,181134
2,90,otro,82.0,7136,pave,otro,otro,inside,gtl,norm,norm,otro,2story,6,6,1946,1950,gable,compshg,brkface,423.0,2,2,cblock,3.0,2.0,no,otro,484,unf,0,495,979,gasa,2,n,otro,979,979,0,1958,0,0,2,0,4,2,2,8,typ,0,attchd,1946.0,unf,2,492,2.0,2.0,y,0,0,0,0,0,0,0,145000
3,60,rl,79.0,9245,pave,otro,lvl,inside,gtl,norm,norm,1fam,2story,7,5,2006,2006,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,otro,unf,0,unf,0,939,939,gasa,3,y,sbrkr,939,858,0,1797,0,0,2,1,3,1,3,8,typ,0,attchd,2006.0,rfn,2,639,2.0,2.0,y,144,53,0,0,0,0,0,213500
4,60,otro,65.0,8125,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,7,5,2005,2006,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,no,unf,0,unf,0,756,756,gasa,3,y,sbrkr,756,797,0,1553,0,0,2,1,3,1,3,6,typ,0,attchd,2005.0,rfn,2,615,2.0,2.0,y,0,45,0,0,0,0,0,186500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,90,rm,68.0,8930,pave,reg,lvl,inside,gtl,otro,norm,otro,1 5fin,6,5,1978,1978,gable,compshg,none,0.0,2,2,otro,2.0,2.0,no,unf,0,unf,0,0,0,gasa,2,y,sbrkr,1318,584,0,1902,0,0,2,0,4,2,2,8,typ,0,attchd,1978.0,unf,2,539,2.0,2.0,y,0,0,0,0,0,0,0,112000
188,120,rl,48.0,6240,pave,reg,lvl,inside,gtl,norm,norm,otro,1story,8,5,2006,2006,hip,compshg,brkface,176.0,3,2,pconc,3.0,2.0,no,glq,863,unf,0,461,1324,gasa,3,y,sbrkr,1324,0,0,1324,1,0,2,0,2,1,3,6,typ,1,attchd,2006.0,fin,2,550,2.0,2.0,y,192,38,0,0,0,0,0,254000
189,60,rl,88.0,14200,pave,reg,lvl,corner,gtl,norm,norm,1fam,2story,7,6,1966,1966,gable,compshg,brkface,309.0,2,2,cblock,2.0,2.0,no,otro,445,unf,0,479,924,gasa,3,y,sbrkr,1216,941,0,2157,0,0,2,1,4,1,3,8,typ,2,attchd,1966.0,fin,2,487,2.0,2.0,y,105,66,0,0,0,0,0,226000
190,20,rl,73.0,8688,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,7,5,2005,2005,gable,compshg,brkface,228.0,3,2,pconc,3.0,2.0,otro,unf,0,unf,0,1616,1616,gasa,3,y,sbrkr,1616,0,0,1616,0,0,2,0,3,1,3,7,typ,0,attchd,2005.0,rfn,3,834,2.0,2.0,y,208,59,0,0,0,0,0,232000


## Reduccion de dimensiones

In [74]:
X_train["v_GarageYrBlt"]=X_train["v_GarageYrBlt"].astype(int)
X_test["v_GarageYrBlt"]=X_test["v_GarageYrBlt"].astype(int)
X_train["v_YearRemodAdd"]=X_train["v_YearRemodAdd"].astype(int)
X_test["v_YearRemodAdd"]=X_test["v_YearRemodAdd"].astype(int)

In [75]:
X_train

,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,c_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal,tgt_SalePrice
0,20,rl,66.0,7742,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,7,1966,1966,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,no,otro,763,unf,0,192,955,gasa,3,y,sbrkr,955,0,0,955,1,0,1,0,3,1,2,6,typ,0,attchd,1966,unf,1,386,2.0,2.0,y,0,0,0,0,0,0,0,127000
1,190,rm,33.0,4456,pave,reg,lvl,inside,gtl,norm,norm,otro,2story,4,5,1920,2008,gable,compshg,none,0.0,2,2,otro,2.0,2.0,no,unf,0,unf,0,736,736,gasa,3,y,sbrkr,736,716,0,1452,0,0,2,0,2,3,2,8,typ,0,attchd,1920,unf,0,0,2.0,2.0,n,0,0,102,0,0,0,0,113000
2,20,rl,60.0,9600,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,7,1950,1995,gable,compshg,none,0.0,2,2,cblock,3.0,2.0,no,otro,442,unf,0,625,1067,gasa,2,y,sbrkr,1067,0,0,1067,0,0,2,0,2,1,3,4,otro,0,attchd,1996,unf,2,436,2.0,2.0,y,290,0,0,0,0,0,0,128000
3,60,rl,64.0,8633,pave,reg,lvl,otro,gtl,norm,norm,1fam,2story,6,5,2005,2005,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,no,glq,193,unf,0,545,738,gasa,3,y,sbrkr,738,738,0,1476,1,0,2,1,3,1,3,7,typ,0,attchd,2005,fin,2,540,2.0,2.0,y,100,35,0,0,0,0,0,173500
4,30,rl,60.0,5400,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,5,6,1920,1950,gable,compshg,none,0.0,2,2,otro,1.0,2.0,no,unf,0,unf,0,691,691,gasa,3,y,fusea,691,0,0,691,0,0,1,0,2,1,3,4,typ,0,detchd,1920,unf,1,216,1.0,2.0,n,0,20,94,0,0,0,0,86000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,70,rl,55.0,10592,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,6,7,1923,1996,hip,compshg,none,0.0,2,3,pconc,2.0,1.0,no,unf,0,unf,0,602,602,gasa,2,y,sbrkr,900,602,0,1502,0,0,1,1,3,1,3,7,typ,2,detchd,1923,unf,1,180,2.0,2.0,y,96,0,112,0,53,0,0,165000
760,60,rl,74.0,8834,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,9,5,2004,2005,hip,compshg,otro,216.0,3,2,pconc,3.0,2.0,no,glq,1170,unf,0,292,1462,gasa,3,y,sbrkr,1462,762,0,2224,1,0,2,1,4,1,3,10,typ,1,attchd,2004,fin,3,738,2.0,2.0,y,184,0,0,0,0,0,0,350000
761,20,rl,85.0,10182,pave,otro,lvl,otro,gtl,otro,norm,1fam,1story,8,5,2006,2006,hip,compshg,otro,420.0,3,2,pconc,3.0,2.0,otro,glq,1220,unf,0,440,1660,gasa,3,y,sbrkr,1660,0,0,1660,1,0,2,0,3,1,3,8,typ,1,attchd,2006,rfn,2,500,2.0,2.0,y,322,50,0,0,0,0,0,290000
762,50,rl,60.0,9738,pave,reg,lvl,inside,gtl,norm,norm,1fam,1 5fin,5,7,1924,1950,gable,compshg,none,0.0,2,3,otro,2.0,2.0,no,otro,392,unf,0,392,784,gasa,3,y,sbrkr,949,272,0,1221,1,0,1,0,4,1,2,7,typ,0,attchd,1965,unf,1,392,2.0,2.0,y,0,0,236,0,0,0,0,104900


### Relación de valor perdido

In [76]:
completitud(X_train)

,columna,total,completitud
0,v_MSSubClass,0,100.0
1,v_CentralAir,0,100.0
2,v_Electrical,0,100.0
3,c_1stFlrSF,0,100.0
4,c_2ndFlrSF,0,100.0
...,...,...,...
62,c_BsmtFinSF1,0,100.0
63,v_BsmtFinType2,0,100.0
64,c_BsmtFinSF2,0,100.0
65,v_YearRemodAdd,0,100.0


### Filtro de baja varianza

In [77]:
X_train.filter(like="c_").describe(percentiles=np.arange(0.1,1.1,.1))

,c_LotFrontage,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,c_1stFlrSF,c_2ndFlrSF,c_GrLivArea,c_BsmtFullBath,c_HalfBath,c_BedroomAbvGr,c_TotRmsAbvGrd,c_GarageCars,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal
count,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000
mean,66.581152,8872.678010,72.240838,367.835079,29.321990,574.729058,971.886126,1064.575916,310.926702,1380.277487,0.346859,0.327225,2.769634,6.191099,1.678010,443.714660,75.806283,35.836387,21.349476,2.490838,7.942408,0.753927,9.128272
std,20.488400,3352.592358,133.421394,379.021634,112.953213,409.986630,346.485018,301.905687,382.102139,389.107954,0.476282,0.472291,0.680621,1.345883,0.697111,192.095732,101.519655,49.148670,55.809187,20.894635,37.206474,20.838949,64.848401
min,21.000000,1300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,438.000000,0.000000,438.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10%,43.000000,4410.500000,0.000000,0.000000,0.000000,102.000000,604.700000,728.000000,0.000000,883.200000,0.000000,0.000000,2.000000,5.000000,1.000000,240.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20%,51.000000,6240.000000,0.000000,0.000000,0.000000,191.600000,728.000000,810.000000,0.000000,1037.000000,0.000000,0.000000,2.000000,5.000000,1.000000,286.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30%,60.000000,7500.000000,0.000000,0.000000,0.000000,294.000000,793.000000,864.000000,0.000000,1131.900000,0.000000,0.000000,2.000000,5.000000,1.000000,336.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
40%,60.000000,8400.000000,0.000000,96.000000,0.000000,403.200000,864.000000,932.000000,0.000000,1235.200000,0.000000,0.000000,3.000000,6.000000,2.000000,404.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,65.000000,8939.500000,0.000000,325.000000,0.000000,522.500000,924.500000,1012.500000,0.000000,1357.500000,0.000000,0.000000,3.000000,6.000000,2.000000,450.500000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000
60%,70.000000,9583.800000,0.000000,442.000000,0.000000,635.800000,1008.000000,1090.600000,425.600000,1484.000000,0.000000,0.000000,3.000000,6.000000,2.000000,486.400000,73.600000,35.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [78]:
X_train.drop(columns=["c_BsmtFullBath","c_PoolArea"],inplace=True)
X_test.drop(columns=["c_BsmtFullBath","c_PoolArea"],inplace=True)

### Filtro de alta correlación

- La Correlación se utiliza para probar las relaciones entre variables cuantitativas o variables categóricas. En otras palabras, es una medida de cómo se relacionan las cosas. El estudio de cómo se correlacionan las variables se denomina análisis de correlación.
- Un coeficiente de correlación proporciona un resumen numérico del grado de asociación entre dos variables.
- Tipos:
- **Correlación r de Pearson: la correlación r de Pearson es el estadístico de correlación más utilizado para medir el grado de relación entre variables relacionadas linealmente. Por ejemplo, en el mercado de valores, si queremos medir cómo se relacionan dos acciones entre sí, se utiliza la correlación r de Pearson para medir el grado de relación entre las dos. La correlación punto-biserial se realiza con la fórmula de correlación de Pearson excepto que una de las variables es dicotómica.**
- **Correlación de rango de Kendall: la correlación de rango de Kendall es una prueba no paramétrica que mide la fuerza de la dependencia entre dos variables. Si consideramos dos muestras, ayb, donde cada tamaño de muestra es n, sabemos que el número total de emparejamientos con a b es n (n-1) / 2.**
- **Correlación de rango de Spearman: la correlación de rango de Spearman es una prueba no paramétrica que se utiliza para medir el grado de asociación entre dos variables. La prueba de correlación de rango de Spearman no conlleva ninguna suposición sobre la distribución de los datos y es el análisis de correlación apropiado cuando las variables se miden en una escala que es al menos ordinal.**

https://www.google.com/search?q=correlation+between+categorical+and+numerical+variables&rlz=1C1SQJL_esMX974MX980&sxsrf=AOaemvKEG-VYi6gPeh-rCa-8uohkKvAAGw:1642729480236&source=lnms&tbm=isch&sa=X&ved=2ahUKEwj0goaZ3MH1AhUoKEQIHULoBYoQ_AUoAXoECAEQAw&biw=1745&bih=881&dpr=1.1#imgrc=r8MNSNFhNHJGmM

In [79]:
X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

- GrLivArea: superficie habitable por encima del nivel del suelo en pies cuadrados
- TotRmsAbvGrd: total de habitaciones por encima del nivel del suelo (no incluye baños)
- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados

In [80]:
corr=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman")

In [81]:
X_train.filter(like="v_").columns

Index(['v_MSSubClass', 'v_MSZoning', 'v_Street', 'v_LotShape', 'v_LandContour',
       'v_LotConfig', 'v_LandSlope', 'v_Condition1', 'v_Condition2',
       'v_BldgType', 'v_HouseStyle', 'v_OverallQual', 'v_OverallCond',
       'v_YearBuilt', 'v_YearRemodAdd', 'v_RoofStyle', 'v_RoofMatl',
       'v_MasVnrType', 'v_ExterQual', 'v_ExterCond', 'v_Foundation',
       'v_BsmtQual', 'v_BsmtCond', 'v_BsmtExposure', 'v_BsmtFinType1',
       'v_BsmtFinType2', 'v_Heating', 'v_HeatingQC', 'v_CentralAir',
       'v_Electrical', 'v_LowQualFinSF', 'v_BsmtHalfBath', 'v_FullBath',
       'v_KitchenAbvGr', 'v_KitchenQual', 'v_Functional', 'v_Fireplaces',
       'v_GarageType', 'v_GarageYrBlt', 'v_GarageFinish', 'v_GarageQual',
       'v_GarageCond', 'v_PavedDrive'],
      dtype='object')

In [82]:
corr=abs(corr)

In [83]:
for col in corr.columns:
    display(corr[[col]][corr[[col]]>.8].dropna())

,c_LotFrontage
c_LotFrontage,1.0


,c_LotArea
c_LotArea,1.0


,c_MasVnrArea
c_MasVnrArea,1.0


,c_BsmtFinSF1
c_BsmtFinSF1,1.0


,c_BsmtFinSF2
c_BsmtFinSF2,1.0


...

,c_OpenPorchSF
c_OpenPorchSF,1.0


,c_EnclosedPorch
c_EnclosedPorch,1.0


,c_3SsnPorch
c_3SsnPorch,1.0


,c_ScreenPorch
c_ScreenPorch,1.0


,c_MiscVal
c_MiscVal,1.0


In [84]:
X_train.drop(columns=["c_TotRmsAbvGrd","c_GarageCars"],inplace=True)
X_test.drop(columns=["c_TotRmsAbvGrd","c_GarageCars"],inplace=True)

In [85]:
corr=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman")
corr=abs(corr)

In [86]:
X_train.drop(columns=["c_1stFlrSF"],inplace=True)
X_test.drop(columns=["c_1stFlrSF"],inplace=True)

In [87]:
for col in corr.columns:
    display(corr[[col]][corr[[col]]>.8].dropna())

,c_LotFrontage
c_LotFrontage,1.0


,c_LotArea
c_LotArea,1.0


,c_MasVnrArea
c_MasVnrArea,1.0


,c_BsmtFinSF1
c_BsmtFinSF1,1.0


,c_BsmtFinSF2
c_BsmtFinSF2,1.0


...

,c_OpenPorchSF
c_OpenPorchSF,1.0


,c_EnclosedPorch
c_EnclosedPorch,1.0


,c_3SsnPorch
c_3SsnPorch,1.0


,c_ScreenPorch
c_ScreenPorch,1.0


,c_MiscVal
c_MiscVal,1.0


In [88]:
X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

### Correlación (con el objetivo)

In [89]:
feats=list(X_train.filter(like="c_").columns)+["tgt_SalePrice"]

In [90]:
X_train[feats].corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

In [91]:
corr=X_train[feats].corr(method="spearman")

In [92]:
corr=abs(corr)

In [93]:
corr[["tgt_SalePrice"]].sort_values(by = 'tgt_SalePrice',ascending = False).style.background_gradient()

,tgt_SalePrice
tgt_SalePrice,1.000000
c_GrLivArea,0.711412
c_GarageArea,0.626417
c_TotalBsmtSF,0.542406
c_OpenPorchSF,0.505586
c_MasVnrArea,0.399387
c_WoodDeckSF,0.382732
c_LotArea,0.373569
c_HalfBath,0.363060
c_LotFrontage,0.331040


In [94]:
corr=corr[["tgt_SalePrice"]].sort_values(by = 'tgt_SalePrice',ascending = False)

In [95]:
cols_drop_low_corr=list(corr[corr["tgt_SalePrice"]<.1].index)

In [96]:
cols_drop_low_corr

['c_MiscVal', 'c_ScreenPorch', 'c_BsmtFinSF2', 'c_3SsnPorch']

- 3SsnPorch: área de porche de tres estaciones en pies cuadrados
- ScreenPorch: superficie acristalada del porche en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados

In [97]:
X_train=X_train.drop(columns=cols_drop_low_corr)
X_test=X_test.drop(columns=cols_drop_low_corr)

### MULTICOLINEALIDAD

#### MULTICOLINEALIDAD CONTINUAS

In [98]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [99]:
#Función para multicolineaidad con continuas
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [100]:
X_train.drop(columns=['v_YearBuilt','v_YearRemodAdd','v_GarageYrBlt'],inplace=True)
X_test.drop(columns=['v_YearBuilt','v_YearRemodAdd','v_GarageYrBlt'],inplace=True)

In [101]:
for col in X_train.filter(like="c_").columns:
    X_train[col]=X_train[col].astype(float)

In [102]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

In [103]:
vif

,variables,VIF
0,c_LotFrontage,18.810579
1,c_LotArea,14.722733
2,c_MasVnrArea,1.575947
3,c_BsmtFinSF1,24.856829
4,c_BsmtUnfSF,41.188090
5,c_TotalBsmtSF,108.747788
6,c_2ndFlrSF,9.157655
7,c_GrLivArea,67.771742
8,c_HalfBath,2.918473
9,c_BedroomAbvGr,22.749393


In [104]:
vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False).head(50)

,variables,VIF
5,c_TotalBsmtSF,108.747788
0,c_LotFrontage,18.810579
1,c_LotArea,14.722733
7,c_GrLivArea,67.771742
4,c_BsmtUnfSF,41.188090
3,c_BsmtFinSF1,24.856829
9,c_BedroomAbvGr,22.749393


In [105]:
vif_feats=list(vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)["variables"].values)

In [106]:
vif_feats

['c_TotalBsmtSF',
 'c_LotFrontage',
 'c_LotArea',
 'c_GrLivArea',
 'c_BsmtUnfSF',
 'c_BsmtFinSF1',
 'c_BedroomAbvGr']

In [107]:
X_train.drop(columns=["c_TotalBsmtSF"],inplace=True)
X_test.drop(columns=["c_TotalBsmtSF"],inplace=True)

In [108]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in ["tgt_SalePrice"]]])

In [109]:
vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
0,c_LotFrontage,18.731596
1,c_LotArea,14.637984
6,c_GrLivArea,63.480470
8,c_BedroomAbvGr,22.451360


In [110]:
vif_feats=["c_GrLivArea"]

In [111]:
X_train.drop(columns=vif_feats,inplace=True)
X_test.drop(columns=vif_feats,inplace=True)

In [112]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

vif[vif["VIF"]>7].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
0,c_LotFrontage,18.395328
1,c_LotArea,14.519405
8,c_GarageArea,8.817986
7,c_BedroomAbvGr,18.205078


In [113]:
vif_feats=list(vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)["variables"].values)

In [114]:
vif_feats

['c_LotFrontage', 'c_LotArea', 'c_BedroomAbvGr']

In [115]:
X_train.drop(columns=["c_LotFrontage"],inplace=True)
X_test.drop(columns=["c_LotFrontage"],inplace=True)

In [116]:
X_train.columns

Index(['v_MSSubClass', 'v_MSZoning', 'c_LotArea', 'v_Street', 'v_LotShape',
       'v_LandContour', 'v_LotConfig', 'v_LandSlope', 'v_Condition1',
       'v_Condition2', 'v_BldgType', 'v_HouseStyle', 'v_OverallQual',
       'v_OverallCond', 'v_RoofStyle', 'v_RoofMatl', 'v_MasVnrType',
       'c_MasVnrArea', 'v_ExterQual', 'v_ExterCond', 'v_Foundation',
       'v_BsmtQual', 'v_BsmtCond', 'v_BsmtExposure', 'v_BsmtFinType1',
       'c_BsmtFinSF1', 'v_BsmtFinType2', 'c_BsmtUnfSF', 'v_Heating',
       'v_HeatingQC', 'v_CentralAir', 'v_Electrical', 'c_2ndFlrSF',
       'v_LowQualFinSF', 'v_BsmtHalfBath', 'v_FullBath', 'c_HalfBath',
       'c_BedroomAbvGr', 'v_KitchenAbvGr', 'v_KitchenQual', 'v_Functional',
       'v_Fireplaces', 'v_GarageType', 'v_GarageFinish', 'c_GarageArea',
       'v_GarageQual', 'v_GarageCond', 'v_PavedDrive', 'c_WoodDeckSF',
       'c_OpenPorchSF', 'c_EnclosedPorch', 'tgt_SalePrice'],
      dtype='object')

In [117]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

vif_feats=list(vif[vif["VIF"]>7].sort_values(by=["VIF"],ascending=False)["variables"][:10].values)

In [118]:


vif[vif["VIF"]>7].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
0,c_LotArea,10.042730
7,c_GarageArea,8.623015
6,c_BedroomAbvGr,15.923777


In [119]:
X_train.drop(columns=["c_BedroomAbvGr"],inplace=True)
X_test.drop(columns=["c_BedroomAbvGr"],inplace=True)

In [120]:
#BsmtCond: Estado general del sótano
#ExterCond: Estado actual del material en el exterior

In [121]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])
vif[vif["VIF"]>7].sort_values(by=["VIF"],ascending=False)

,variables,VIF
6,c_GarageArea,8.557806
0,c_LotArea,7.063067


In [122]:
vif

,variables,VIF
0,c_LotArea,7.063067
1,c_MasVnrArea,1.562610
2,c_BsmtFinSF1,3.813029
3,c_BsmtUnfSF,5.070706
4,c_2ndFlrSF,3.438439
5,c_HalfBath,2.896010
6,c_GarageArea,8.557806
7,c_WoodDeckSF,1.741656
8,c_OpenPorchSF,1.811682
9,c_EnclosedPorch,1.232342


#### MULTICOLINEALIDAD CATEGORICAS

In [123]:
X_train

,v_MSSubClass,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,tgt_SalePrice
0,20,rl,7742.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,7,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,no,otro,763.0,unf,192.0,gasa,3,y,sbrkr,0.0,0,0,1,0.0,1,2,typ,0,attchd,unf,386.0,2.0,2.0,y,0.0,0.0,0.0,127000
1,190,rm,4456.0,pave,reg,lvl,inside,gtl,norm,norm,otro,2story,4,5,gable,compshg,none,0.0,2,2,otro,2.0,2.0,no,unf,0.0,unf,736.0,gasa,3,y,sbrkr,716.0,0,0,2,0.0,3,2,typ,0,attchd,unf,0.0,2.0,2.0,n,0.0,0.0,102.0,113000
2,20,rl,9600.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,4,7,gable,compshg,none,0.0,2,2,cblock,3.0,2.0,no,otro,442.0,unf,625.0,gasa,2,y,sbrkr,0.0,0,0,2,0.0,1,3,otro,0,attchd,unf,436.0,2.0,2.0,y,290.0,0.0,0.0,128000
3,60,rl,8633.0,pave,reg,lvl,otro,gtl,norm,norm,1fam,2story,6,5,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,no,glq,193.0,unf,545.0,gasa,3,y,sbrkr,738.0,0,0,2,1.0,1,3,typ,0,attchd,fin,540.0,2.0,2.0,y,100.0,35.0,0.0,173500
4,30,rl,5400.0,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,5,6,gable,compshg,none,0.0,2,2,otro,1.0,2.0,no,unf,0.0,unf,691.0,gasa,3,y,fusea,0.0,0,0,1,0.0,1,3,typ,0,detchd,unf,216.0,1.0,2.0,n,0.0,20.0,94.0,86000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,70,rl,10592.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,6,7,hip,compshg,none,0.0,2,3,pconc,2.0,1.0,no,unf,0.0,unf,602.0,gasa,2,y,sbrkr,602.0,0,0,1,1.0,1,3,typ,2,detchd,unf,180.0,2.0,2.0,y,96.0,0.0,112.0,165000
760,60,rl,8834.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,9,5,hip,compshg,otro,216.0,3,2,pconc,3.0,2.0,no,glq,1170.0,unf,292.0,gasa,3,y,sbrkr,762.0,0,0,2,1.0,1,3,typ,1,attchd,fin,738.0,2.0,2.0,y,184.0,0.0,0.0,350000
761,20,rl,10182.0,pave,otro,lvl,otro,gtl,otro,norm,1fam,1story,8,5,hip,compshg,otro,420.0,3,2,pconc,3.0,2.0,otro,glq,1220.0,unf,440.0,gasa,3,y,sbrkr,0.0,0,0,2,0.0,1,3,typ,1,attchd,rfn,500.0,2.0,2.0,y,322.0,50.0,0.0,290000
762,50,rl,9738.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1 5fin,5,7,gable,compshg,none,0.0,2,3,otro,2.0,2.0,no,otro,392.0,unf,392.0,gasa,3,y,sbrkr,272.0,0,0,1,0.0,1,2,typ,0,attchd,unf,392.0,2.0,2.0,y,0.0,0.0,236.0,104900


In [124]:
## Multicolinealidad Categoricas##
def multicolinealidad_cate(df,y,threshold=5,only_final_vif=False):
    '''
    Función para calcular el VIF Genralizado, (GVIF, Fox and Monette 1992)
    Las variables no deben de deben de estar transformadas con OneHotEncoder ya que la función lo realiza internamnete.
    Args:
        df
        y
        threshold
        only_final_vif
    
    Returns:
        pandas data frame: a data frame, indexed by factor of the GVIF, GVIF^(1/2Df), VIF^(1/2Df)^2 
    '''

    df_x = df.drop(y,axis=1)
    #Guardamos los nombres de las columnas añadiendo el prefijo

    onehot_list = list(df_x.select_dtypes(include=['category', 'object', 'string']).columns)
    #Generamos las variables dummy 
    
    df_1hot = pd.get_dummies(df_x, drop_first=True, dummy_na=False, prefix_sep='_')


    #Dataframe vacio para guardar los resultados
    gvif_df = pd.DataFrame(columns = ['factor','GVIF','Df','GVIF^(1/2Df)', 'VIF'])

    # Iteramos sobre las columas
    for (columnName, columnData) in df_x.iteritems():
        #Si se generaron las de una columna, es decir la variable tiene más de dos posibilidades utilizamos todas las variables creadas
        
        if columnName in onehot_list:
            X1 = df_1hot.loc[:, df_1hot.columns.str.startswith(columnName)]
            X2 = df_1hot.loc[:, ~df_1hot.columns.str.startswith(columnName)]
        else:
            X1 = df_1hot[[columnName]].values
            X2 = df_1hot.loc[:, df_1hot.columns != columnName].values
        #display(X1)
        # Calculamose gvif
        #A la matriz de correlación de las variables codificadas en caliente del atributo en consideración.
        #B la matriz de correlación de todos los demás atributos del conjunto de datos (uno codificado en dummy y numérico) excluyendo los de A.
        #C la matriz de correlación de las variables consideradas tanto en A como en B.
        #GVIF= (det(A)*det(B))/det(C)
        #display(np.corrcoef(X1, rowvar=False))
        gvif = np.linalg.det(np.array(np.corrcoef(X1, rowvar=False), ndmin=2)) * np.linalg.det(np.corrcoef(X2, rowvar=False)) / np.linalg.det(np.corrcoef(np.append(X1, X2, axis=1), rowvar=False))
        #(GVIF) elevado a (1 / (2 * grados de libertad))
        #Los grados de libertad es 1-n (n=Número de opciones que tiene la variable categorica)
        gvif_12df = np.power(gvif, 1 / (2 * X1.shape[1]))
        gvif_12df_sq = gvif_12df ** 2
        DF =  X1.shape[1]
    

        # 
        new_row = {'factor': columnName, 'GVIF': gvif,'Df':DF, 'GVIF^(1/2Df)': gvif_12df, 'VIF': gvif_12df_sq}
        gvif_df = gvif_df.append(new_row, ignore_index=True)

    gvif_df = gvif_df.set_index('factor')
    if only_final_vif:
        gvif_df_final = gvif_df.drop(['GVIF','Df','GVIF^(1/2Df)'],axis=1)
    else:
        gvif_df_final = gvif_df 
    gvif_filter = gvif_df.loc[gvif_df['VIF'] >= threshold]['VIF'].to_dict()
    if gvif_filter:
        for i in gvif_filter.keys():
            df_x_m = df_x.drop([i],axis=1)
    else:
        df_x_m = df_x
    df_m=pd.concat([df_x_m,df[y]],axis=1)
    return gvif_df_final


In [125]:
v_feats=list(X_train.filter(like="v_").columns)

In [126]:
results=multicolinealidad_cate(df=X_train[v_feats+["tgt_SalePrice"]],y="tgt_SalePrice")

In [127]:
X_train["v_BldgType"].value_counts()

1fam    640
otro    124
Name: v_BldgType, dtype: int64

In [128]:
X_train["v_OverallQual"].value_counts()

5     234
6     192
7     169
8      75
4      69
3      12
9       8
2       3
10      2
Name: v_OverallQual, dtype: int64

In [129]:
X_train["v_OverallCond"].value_counts()

5    428
6    133
7    111
8     41
4     31
9      9
3      9
2      2
Name: v_OverallCond, dtype: int64

In [130]:
X_train[["v_OverallQual","v_OverallCond"]].head(10)

,v_OverallQual,v_OverallCond
0,5,7
1,4,5
2,4,7
3,6,5
4,5,6
5,5,7
6,8,6
7,5,6
8,5,5
9,5,5


In [131]:
results

,GVIF,Df,GVIF^(1/2Df),VIF
factor,,,,
v_MSSubClass,17.649673,1,4.201151,17.649673
v_MSZoning,2.319643,2,1.234114,1.523037
v_Street,1.327346,1,1.152105,1.327346
v_LotShape,1.426086,1,1.194188,1.426086
v_LandContour,1.579098,1,1.256622,1.579098
v_LotConfig,1.319852,2,1.071843,1.148848
v_LandSlope,1.491914,1,1.221439,1.491914
v_Condition1,1.308398,1,1.143852,1.308398
v_Condition2,1.508984,1,1.228407,1.508984


In [132]:
X_train=X_train.drop(columns=["v_MSSubClass","v_OverallQual"])
X_test=X_test.drop(columns=["v_MSSubClass","v_OverallQual"])

In [133]:
v_feats=list(X_train.filter(like="v_").columns)

In [134]:
results=multicolinealidad_cate(df=X_train[v_feats+["tgt_SalePrice"]],y="tgt_SalePrice")

In [135]:
results

,GVIF,Df,GVIF^(1/2Df),VIF
factor,,,,
v_MSZoning,2.245270,2,1.224101,1.498422
v_Street,1.265480,1,1.124935,1.265480
v_LotShape,1.405069,1,1.185356,1.405069
v_LandContour,1.523022,1,1.234108,1.523022
v_LotConfig,1.269094,2,1.061386,1.126541
v_LandSlope,1.462862,1,1.209488,1.462862
v_Condition1,1.283916,1,1.133100,1.283916
v_Condition2,1.468170,1,1.211681,1.468170
v_BldgType,1.737584,1,1.318175,1.737584


In [136]:
X_train

,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,tgt_SalePrice
0,rl,7742.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,7,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,no,otro,763.0,unf,192.0,gasa,3,y,sbrkr,0.0,0,0,1,0.0,1,2,typ,0,attchd,unf,386.0,2.0,2.0,y,0.0,0.0,0.0,127000
1,rm,4456.0,pave,reg,lvl,inside,gtl,norm,norm,otro,2story,5,gable,compshg,none,0.0,2,2,otro,2.0,2.0,no,unf,0.0,unf,736.0,gasa,3,y,sbrkr,716.0,0,0,2,0.0,3,2,typ,0,attchd,unf,0.0,2.0,2.0,n,0.0,0.0,102.0,113000
2,rl,9600.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,7,gable,compshg,none,0.0,2,2,cblock,3.0,2.0,no,otro,442.0,unf,625.0,gasa,2,y,sbrkr,0.0,0,0,2,0.0,1,3,otro,0,attchd,unf,436.0,2.0,2.0,y,290.0,0.0,0.0,128000
3,rl,8633.0,pave,reg,lvl,otro,gtl,norm,norm,1fam,2story,5,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,no,glq,193.0,unf,545.0,gasa,3,y,sbrkr,738.0,0,0,2,1.0,1,3,typ,0,attchd,fin,540.0,2.0,2.0,y,100.0,35.0,0.0,173500
4,rl,5400.0,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,6,gable,compshg,none,0.0,2,2,otro,1.0,2.0,no,unf,0.0,unf,691.0,gasa,3,y,fusea,0.0,0,0,1,0.0,1,3,typ,0,detchd,unf,216.0,1.0,2.0,n,0.0,20.0,94.0,86000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,rl,10592.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,7,hip,compshg,none,0.0,2,3,pconc,2.0,1.0,no,unf,0.0,unf,602.0,gasa,2,y,sbrkr,602.0,0,0,1,1.0,1,3,typ,2,detchd,unf,180.0,2.0,2.0,y,96.0,0.0,112.0,165000
760,rl,8834.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,5,hip,compshg,otro,216.0,3,2,pconc,3.0,2.0,no,glq,1170.0,unf,292.0,gasa,3,y,sbrkr,762.0,0,0,2,1.0,1,3,typ,1,attchd,fin,738.0,2.0,2.0,y,184.0,0.0,0.0,350000
761,rl,10182.0,pave,otro,lvl,otro,gtl,otro,norm,1fam,1story,5,hip,compshg,otro,420.0,3,2,pconc,3.0,2.0,otro,glq,1220.0,unf,440.0,gasa,3,y,sbrkr,0.0,0,0,2,0.0,1,3,typ,1,attchd,rfn,500.0,2.0,2.0,y,322.0,50.0,0.0,290000
762,rl,9738.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1 5fin,7,gable,compshg,none,0.0,2,3,otro,2.0,2.0,no,otro,392.0,unf,392.0,gasa,3,y,sbrkr,272.0,0,0,1,0.0,1,2,typ,0,attchd,unf,392.0,2.0,2.0,y,0.0,0.0,236.0,104900


In [137]:
X_train["v_LowQualFinSF"].value_counts()

0      752
80       3
360      2
390      1
144      1
234      1
420      1
513      1
514      1
473      1
Name: v_LowQualFinSF, dtype: int64

In [138]:
##CREAR LAS VARIABLES DUMMY

In [139]:
from sklearn.preprocessing import OneHotEncoder

In [140]:
dummy_vars=["v_MSZoning","v_Street","v_LotShape","v_LandContour","v_LotConfig","v_LandSlope","v_Condition1","v_Condition2","v_BldgType","v_HouseStyle",
"v_RoofStyle","v_MasVnrType","v_Foundation","v_BsmtExposure","v_BsmtFinType1","v_BsmtFinType2","v_Heating","v_CentralAir",
"v_Electrical","v_Functional","v_GarageType","v_GarageFinish","v_PavedDrive","v_RoofMatl","v_OverallCond"]

In [141]:
X_train.shape

(764, 49)

In [142]:
X_test.shape

(192, 49)

In [143]:
X_test

,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,tgt_SalePrice
0,rl,8816,pave,reg,lvl,inside,gtl,norm,norm,1fam,otro,7,gable,compshg,brkface,80.0,2,2,cblock,2.0,2.0,otro,glq,504,unf,506,gasa,3,y,sbrkr,0,0,0,1,0,1,2,typ,0,attchd,unf,440,2.0,2.0,y,0,0,0,155000
1,rl,8899,pave,otro,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,otro,glq,24,unf,1316,gasa,3,y,sbrkr,0,0,0,2,0,1,3,typ,0,attchd,fin,396,2.0,2.0,y,100,30,0,181134
2,otro,7136,pave,otro,otro,inside,gtl,norm,norm,otro,2story,6,gable,compshg,brkface,423.0,2,2,cblock,3.0,2.0,no,otro,484,unf,495,gasa,2,n,otro,979,0,0,2,0,2,2,typ,0,attchd,unf,492,2.0,2.0,y,0,0,0,145000
3,rl,9245,pave,otro,lvl,inside,gtl,norm,norm,1fam,2story,5,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,otro,unf,0,unf,939,gasa,3,y,sbrkr,858,0,0,2,1,1,3,typ,0,attchd,rfn,639,2.0,2.0,y,144,53,0,213500
4,otro,8125,pave,reg,lvl,inside,gtl,norm,norm,1fam,2story,5,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,no,unf,0,unf,756,gasa,3,y,sbrkr,797,0,0,2,1,1,3,typ,0,attchd,rfn,615,2.0,2.0,y,0,45,0,186500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,rm,8930,pave,reg,lvl,inside,gtl,otro,norm,otro,1 5fin,5,gable,compshg,none,0.0,2,2,otro,2.0,2.0,no,unf,0,unf,0,gasa,2,y,sbrkr,584,0,0,2,0,2,2,typ,0,attchd,unf,539,2.0,2.0,y,0,0,0,112000
188,rl,6240,pave,reg,lvl,inside,gtl,norm,norm,otro,1story,5,hip,compshg,brkface,176.0,3,2,pconc,3.0,2.0,no,glq,863,unf,461,gasa,3,y,sbrkr,0,0,0,2,0,1,3,typ,1,attchd,fin,550,2.0,2.0,y,192,38,0,254000
189,rl,14200,pave,reg,lvl,corner,gtl,norm,norm,1fam,2story,6,gable,compshg,brkface,309.0,2,2,cblock,2.0,2.0,no,otro,445,unf,479,gasa,3,y,sbrkr,941,0,0,2,1,1,3,typ,2,attchd,fin,487,2.0,2.0,y,105,66,0,226000
190,rl,8688,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,brkface,228.0,3,2,pconc,3.0,2.0,otro,unf,0,unf,1616,gasa,3,y,sbrkr,0,0,0,2,0,1,3,typ,0,attchd,rfn,834,2.0,2.0,y,208,59,0,232000


In [144]:
one=OneHotEncoder(drop="first",handle_unknown ="error")
#drop="first"

one.fit(X_train[dummy_vars])

replace_name=dict(zip(list(map(lambda x:f"x{str(x)}_",range(len(dummy_vars)))),dummy_vars))

features=one.get_feature_names()

features=list(map(lambda x:multiple_replace(x, replace_name),features))

X_train[features]=pd.DataFrame(one.transform(X_train[dummy_vars]).toarray(),columns=features)
X_test[features]=pd.DataFrame(one.transform(X_test[dummy_vars]).toarray(),columns=features)

X_train.drop(columns=dummy_vars,inplace=True)
X_test.drop(columns=dummy_vars,inplace=True)

In [145]:
X_train

,c_LotArea,c_MasVnrArea,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,c_BsmtFinSF1,c_BsmtUnfSF,v_HeatingQC,c_2ndFlrSF,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,c_HalfBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,c_GarageArea,v_GarageQual,v_GarageCond,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,tgt_SalePrice,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,7742.0,0.0,2,2,2.0,2.0,763.0,192.0,3,0.0,0,0,1,0.0,1,2,0,386.0,2.0,2.0,0.0,0.0,0.0,127000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,4456.0,0.0,2,2,2.0,2.0,0.0,736.0,3,716.0,0,0,2,0.0,3,2,0,0.0,2.0,2.0,0.0,0.0,102.0,113000,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,9600.0,0.0,2,2,3.0,2.0,442.0,625.0,2,0.0,0,0,2,0.0,1,3,0,436.0,2.0,2.0,290.0,0.0,0.0,128000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,8633.0,0.0,3,2,3.0,2.0,193.0,545.0,3,738.0,0,0,2,1.0,1,3,0,540.0,2.0,2.0,100.0,35.0,0.0,173500,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,5400.0,0.0,2,2,1.0,2.0,0.0,691.0,3,0.0,0,0,1,0.0,1,3,0,216.0,1.0,2.0,0.0,20.0,94.0,86000,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,10592.0,0.0,2,3,2.0,1.0,0.0,602.0,2,602.0,0,0,1,1.0,1,3,2,180.0,2.0,2.0,96.0,0.0,112.0,165000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
760,8834.0,216.0,3,2,3.0,2.0,1170.0,292.0,3,762.0,0,0,2,1.0,1,3,1,738.0,2.0,2.0,184.0,0.0,0.0,350000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,10182.0,420.0,3,2,3.0,2.0,1220.0,440.0,3,0.0,0,0,2,0.0,1,3,1,500.0,2.0,2.0,322.0,50.0,0.0,290000,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
762,9738.0,0.0,2,3,2.0,2.0,392.0,392.0,3,272.0,0,0,1,0.0,1,2,0,392.0,2.0,2.0,0.0,0.0,236.0,104900,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### PCA

In [146]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

**Necesitamos normalizar los datos antes del análisis de componentes principales (PCA)**
- El PCA calcula una nueva proyección de su conjunto de datos. Y el nuevo eje se basa en la desviación estándar de sus variables. Por tanto, una variable con una desviación estándar alta tendrá un peso mayor para el cálculo del eje que una variable con una desviación estándar baja. Si normaliza sus datos, todas las variables tienen la misma desviación estándar, por lo que todas las variables tienen el mismo peso y su PCA calcula el eje relevante.

In [147]:
X_train.shape

(764, 69)

In [148]:
X_train.filter(like="c_")

,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_HalfBath,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch
0,7742.0,0.0,763.0,192.0,0.0,0.0,386.0,0.0,0.0,0.0
1,4456.0,0.0,0.0,736.0,716.0,0.0,0.0,0.0,0.0,102.0
2,9600.0,0.0,442.0,625.0,0.0,0.0,436.0,290.0,0.0,0.0
3,8633.0,0.0,193.0,545.0,738.0,1.0,540.0,100.0,35.0,0.0
4,5400.0,0.0,0.0,691.0,0.0,0.0,216.0,0.0,20.0,94.0
...,...,...,...,...,...,...,...,...,...,...
759,10592.0,0.0,0.0,602.0,602.0,1.0,180.0,96.0,0.0,112.0
760,8834.0,216.0,1170.0,292.0,762.0,1.0,738.0,184.0,0.0,0.0
761,10182.0,420.0,1220.0,440.0,0.0,0.0,500.0,322.0,50.0,0.0
762,9738.0,0.0,392.0,392.0,272.0,0.0,392.0,0.0,0.0,236.0


In [149]:
scaler = StandardScaler()
scaler.fit(X_train.filter(like="c_"))
df_train_sc=pd.DataFrame(scaler.transform(X_train.filter(like="c_")))
df_test_sc=pd.DataFrame(scaler.transform(X_test.filter(like="c_")))

In [150]:
df_train_sc

,0,1,2,3,4,5,6,7,8,9
0,-0.337476,-0.541803,1.043275,-0.934127,-0.814260,-0.693301,-0.300644,-0.747205,-0.729620,-0.382795
1,-1.318255,-0.541803,-0.971121,0.393614,1.060812,-0.693301,-2.311375,-0.747205,-0.729620,1.446059
2,0.217085,-0.541803,0.195803,0.122696,-0.814260,-0.693301,-0.040187,2.111257,-0.729620,-0.382795
3,-0.071537,-0.541803,-0.461582,-0.072560,1.118426,1.425426,0.501565,0.238472,-0.017029,-0.382795
4,-1.036497,-0.541803,-0.971121,0.283783,-0.814260,-0.693301,-1.186199,-0.747205,-0.322425,1.302619
...,...,...,...,...,...,...,...,...,...,...
759,0.513170,-0.541803,-0.971121,0.066560,0.762267,1.425426,-1.373729,0.199045,-0.729620,1.625358
760,-0.011544,1.078188,2.117796,-0.690057,1.181278,1.425426,1.532976,1.066440,-0.729620,-0.382795
761,0.390796,2.608180,2.249801,-0.328833,-0.814260,-0.693301,0.293199,2.426673,0.288368,-0.382795
762,0.258274,-0.541803,0.063798,-0.445987,-0.101942,-0.693301,-0.269389,-0.747205,-0.729620,3.848670


In [151]:
df_train_sc.describe()

,0,1,2,3,4,5,6,7,8,9
count,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02
mean,2.418077e-16,3.603865e-17,-5.580179e-17,4.301388e-17,5.928940e-17,5.115164e-17,6.452081e-17,3.022597e-17,4.766403e-17,3.313231e-17
std,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00
min,-2.260233e+00,-5.418034e-01,-9.711215e-01,-1.402742e+00,-8.142597e-01,-6.933006e-01,-2.311375e+00,-7.472045e-01,-7.296202e-01,-3.827947e-01
25%,-5.134997e-01,-5.418034e-01,-9.711215e-01,-7.974482e-01,-8.142597e-01,-6.933006e-01,-7.486309e-01,-7.472045e-01,-7.296202e-01,-3.827947e-01
50%,1.994450e-02,-5.418034e-01,-1.130889e-01,-1.274756e-01,-8.142597e-01,-6.933006e-01,3.534584e-02,-7.472045e-01,-3.631446e-01,-3.827947e-01
75%,5.230191e-01,2.681923e-01,7.211828e-01,5.705652e-01,9.770102e-01,1.425426e+00,5.640744e-01,6.721693e-01,3.901665e-01,-3.827947e-01
max,4.898918e+00,7.925651e+00,3.181756e+00,3.078386e+00,2.718523e+00,3.544153e+00,3.121766e+00,4.269887e+00,5.133990e+00,5.534084e+00


In [155]:
for n_compo in range(1,11):
    print(f"N componentes : {n_compo}")
    pca = PCA(n_components=n_compo)
    pca.fit(df_train_sc)
    print(sum(pca.explained_variance_ratio_))
    print("\n")

N componentes : 1
0.2141259466688928


N componentes : 2
0.3928123786681877


N componentes : 3
0.5336259011276279


N componentes : 4
0.6367704762140114


N componentes : 5
0.7247969429652483


N componentes : 6
0.81082515282018


N componentes : 7
0.8878711982739937


N componentes : 8
0.9467819140643412


N componentes : 9
0.9785717324255463


N componentes : 10
0.9999999999999999




In [156]:
pca = PCA(n_components=8)
pca.fit(df_train_sc)

PCA(n_components=8)

In [157]:
X_train_pca=pd.DataFrame(pca.transform(df_train_sc))
X_test_pca=pd.DataFrame(pca.transform(df_test_sc))

In [158]:
#Porcentaje de varianza explicada por cada uno de los componentes seleccionados.
pca.explained_variance_ratio_

array([0.21412595, 0.17868643, 0.14081352, 0.10314458, 0.08802647,
       0.08602821, 0.07704605, 0.05891072])

In [159]:
sum(pca.explained_variance_ratio_)

0.9467819140643412

In [160]:
X_train.reset_index(drop=True).filter(like="v_")

,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,v_HeatingQC,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,v_GarageQual,v_GarageCond,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,2,2,2.0,2.0,3,0,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,2,2.0,2.0,3,0,0,2,3,2,0,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,2,3.0,2.0,2,0,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,2,3.0,2.0,3,0,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2,2,1.0,2.0,3,0,0,1,1,3,0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,2,3,2.0,1.0,2,0,0,1,1,3,2,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
760,3,2,3.0,2.0,3,0,0,2,1,3,1,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,3,2,3.0,2.0,3,0,0,2,1,3,1,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
762,2,3,2.0,2.0,3,0,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [161]:
pd.concat([X_train_pca,X_train.reset_index(drop=True).filter(like="v_")],axis=1)

,0,1,2,3,4,5,6,7,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,v_HeatingQC,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,v_GarageQual,v_GarageCond,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,-1.272143,1.456640,-0.481700,0.137171,-0.272089,0.486645,0.536046,0.438717,2,2,2.0,2.0,3,0,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-2.313462,-1.867686,-1.417295,0.317701,0.293894,-0.543619,-0.500470,-0.710444,2,2,2.0,2.0,3,0,0,2,3,2,0,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-0.371487,1.179360,0.968352,-1.023342,-0.561778,-1.518224,-0.718625,-0.109079,2,2,3.0,2.0,2,0,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.192986,-0.909516,-0.768962,-0.346530,-0.615444,-0.371474,0.012997,0.857534,3,2,3.0,2.0,3,0,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-2.338012,-0.969983,-0.146232,0.475590,0.308311,0.092722,-0.593544,-0.278899,2,2,1.0,2.0,3,0,0,1,1,3,0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,-0.442620,-1.814469,-1.462102,-1.368695,0.495095,-0.641020,-0.195503,-0.565624,2,3,2.0,1.0,2,0,0,1,1,3,2,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
760,2.523197,1.654720,-1.128297,-0.186969,0.953710,-0.779863,0.025430,1.041829,3,2,3.0,2.0,3,0,0,2,1,3,1,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,1.611685,3.146628,0.824881,0.382335,1.195861,-1.143646,-0.684695,-1.678423,3,2,3.0,2.0,3,0,0,2,1,3,1,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
762,-2.029747,-0.671659,-0.891176,-1.436026,2.629068,1.062625,-1.234496,0.044671,2,3,2.0,2.0,3,0,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [162]:
pd.concat([X_test_pca,X_test.reset_index(drop=True).filter(like="v_")],axis=1)

,0,1,2,3,4,5,6,7,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,v_HeatingQC,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,v_GarageQual,v_GarageCond,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,-0.953557,0.789710,0.294899,0.309496,0.143928,0.188578,0.835322,0.263799,2,2,2.0,2.0,3,0,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.893998,-0.864762,1.830177,0.079425,-0.561311,-0.568350,0.005279,-0.129448,3,2,3.0,2.0,3,0,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.923988,0.180212,-0.693075,1.586843,1.692188,-0.697558,1.205068,-0.260776,2,2,3.0,2.0,2,0,0,2,2,2,0,2.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.776757,-1.660062,0.118523,-0.453224,-0.609008,-0.605753,-0.311162,0.949425,3,2,3.0,2.0,3,0,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.137031,-1.749690,-0.455017,0.174815,-0.475551,0.180806,0.381280,1.289098,3,2,3.0,2.0,3,0,0,2,1,3,0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.546299,-0.098455,-0.656947,-0.292747,-0.291765,0.106258,0.774341,1.036372,2,2,2.0,2.0,2,0,0,2,2,2,0,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
188,0.265089,1.996497,0.521935,0.727141,0.245136,-0.540677,-0.809144,-0.020531,3,2,3.0,2.0,3,0,0,2,1,3,1,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
189,2.795893,-0.369880,-0.702887,-0.395518,0.676486,-0.073613,0.986742,-1.043289,2,2,2.0,2.0,3,0,0,2,1,3,2,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
190,1.133648,-0.510627,3.415327,0.829239,0.703297,-1.060506,-0.656571,0.374589,3,2,3.0,2.0,3,0,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [163]:
#Para visualización creamos dos o tres componentes para tener una idea de como se ve nuestra información continua
#Tomando en cuenta la varianza explicada (algunas veces no es tan representativo)
pca = PCA(n_components=2)
dimention_2=pca.fit_transform(df_train_sc)
total_var = pca.explained_variance_ratio_.sum() * 100
fig = px.scatter(dimention_2, x=0, y=1,title=f'Total de Varianza Explicada: {total_var:.2f}%',
                 labels={'0': 'Componente Principal 1', '1': 'Componente Principal 2'})
fig.show()

In [164]:
pca = PCA(n_components=3)
dimention_3=pca.fit_transform(df_train_sc)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    dimention_3, x=0, y=1, z=2, 
    title=f'Total de Varianza Explicada: {total_var:.2f}%',
    labels={'0': 'CP 1', '1': 'CP 2', '2': 'CP 3'}
)
fig.show()

### K- Best

Score function:
- For regression: f_regression, mutual_info_regression
- For classification: chi2, f_classif, mutual_info_classif

**f_regression**
- Pruebas de regresión lineal univariante.

- Modelo lineal para probar el efecto individual de cada uno de los muchos regresores. Esta es una función de puntuación que se utilizará en un procedimiento de selección de características

- Esto se hace en 2 pasos:

- Se calcula la correlación entre cada regresor y el objetivo, es decir, ((X [:, i] - mean (X [:, i])) * (y - mean_y)) / (std (X [:, i] ) * estándar (y)).

- Se convierte en una puntuación F y luego en un valor p.

- mutual_info_classif : para discretas y continuas
- f_regressión : solo continuas

https://towardsdatascience.com/fisher-test-for-regression-analysis-1e1687867259

** Estime la información mutua para una variable objetivo discreta.

** La información mutua (MI) [1] entre dos variables aleatorias es un valor no negativo, que mide la dependencia entre las variables. Es igual a cero si y solo si dos variables aleatorias son independientes, y los valores más altos significan una mayor dependencia.

** La función se basa en métodos no paramétricos basados ​​en la estimación de la entropía a partir de distancias de los vecinos más cercanos k.

In [914]:
X=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_")
y=X_train[[x for x in X_train if x=="tgt_SalePrice"]]

In [915]:
from sklearn.feature_selection import SelectKBest,mutual_info_classif 

In [920]:
kb = SelectKBest(mutual_info_classif , k=5)

In [921]:
X

,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_HalfBath,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal
0,6240.0,0.0,0.0,816.0,0.0,0.0,528.0,112.0,0.0,0.0,400.0
1,7742.0,0.0,763.0,192.0,0.0,0.0,386.0,0.0,0.0,0.0,0.0
2,9600.0,0.0,442.0,625.0,0.0,0.0,436.0,290.0,0.0,0.0,0.0
3,11600.0,0.0,0.0,700.0,0.0,0.0,252.0,0.0,0.0,67.0,0.0
4,10206.0,0.0,0.0,0.0,0.0,0.0,528.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
759,21286.0,0.0,0.0,720.0,551.0,0.0,312.0,0.0,0.0,108.0,0.0
760,8834.0,216.0,1170.0,292.0,762.0,1.0,738.0,184.0,0.0,0.0,0.0
761,7094.0,0.0,180.0,340.0,0.0,0.0,384.0,0.0,0.0,0.0,0.0
762,2645.0,456.0,0.0,776.0,677.0,1.0,492.0,206.0,0.0,0.0,0.0


In [922]:
X_new=kb.fit_transform(X, y)

In [923]:
cols=X.columns[kb.get_support()]

In [924]:
pd.DataFrame(X_new,columns=cols)

,c_LotArea,c_MasVnrArea,c_HalfBath,c_GarageArea,c_OpenPorchSF
0,6240.0,0.0,0.0,528.0,0.0
1,7742.0,0.0,0.0,386.0,0.0
2,9600.0,0.0,0.0,436.0,0.0
3,11600.0,0.0,0.0,252.0,0.0
4,10206.0,0.0,0.0,528.0,0.0
...,...,...,...,...,...
759,21286.0,0.0,0.0,312.0,0.0
760,8834.0,216.0,1.0,738.0,0.0
761,7094.0,0.0,0.0,384.0,0.0
762,2645.0,456.0,1.0,492.0,0.0


In [925]:
scores=pd.DataFrame(kb.scores_)
scores["Variable"]=X.columns
scores.rename(columns={0:"Score"},inplace=True)

In [926]:
scores=scores.sort_values(by="Score",ascending=False)
scores.reset_index(drop=True,inplace=True)

In [927]:
scores

,Score,Variable
0,0.335697,c_HalfBath
1,0.249196,c_GarageArea
2,0.195370,c_OpenPorchSF
3,0.164919,c_LotArea
4,0.117103,c_MasVnrArea
5,0.074608,c_WoodDeckSF
6,0.065104,c_MiscVal
7,0.055629,c_2ndFlrSF
8,0.044195,c_BsmtUnfSF
9,0.030172,c_BsmtFinSF1


In [928]:
X[list(scores[:20]["Variable"].values)][cols]

,c_LotArea,c_MasVnrArea,c_HalfBath,c_GarageArea,c_OpenPorchSF
0,6240.0,0.0,0.0,528.0,0.0
1,7742.0,0.0,0.0,386.0,0.0
2,9600.0,0.0,0.0,436.0,0.0
3,11600.0,0.0,0.0,252.0,0.0
4,10206.0,0.0,0.0,528.0,0.0
...,...,...,...,...,...
759,21286.0,0.0,0.0,312.0,0.0
760,8834.0,216.0,1.0,738.0,0.0
761,7094.0,0.0,0.0,384.0,0.0
762,2645.0,456.0,1.0,492.0,0.0


## VarClus

- Varclus es un buen algoritmo de reducción de dimensiones. Aquí hay una breve descripción:
- Se elige un grupo para dividir.
- El grupo elegido se divide en dos grupos al encontrar los dos primeros componentes principales, realizar una rotación ortoblicua y asignar cada variable al componente girado con el que tiene la correlación al cuadrado más alto.
- Las variables se reasignan iterativamente a los conglomerados para maximizar la varianza contabilizada por los componentes del conglomerado

In [1126]:
from varclushi import VarClusHi

In [1133]:
vc = VarClusHi(X,maxeigval2=.7)

In [1134]:
vc.varclus()

In [1135]:
vc.rsquare

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,c_EnclosedPorch,1.000000,0.033443,0.000000
1,1,c_BsmtFinSF1,0.786901,0.039508,0.221865
2,1,c_BsmtUnfSF,0.786901,0.037686,0.221445
3,2,c_2ndFlrSF,0.821109,0.031683,0.184744
4,2,c_HalfBath,0.821109,0.037034,0.185771
5,3,c_WoodDeckSF,1.000000,0.043775,0.000000
6,4,c_MiscVal,1.000000,0.004830,0.000000
7,5,c_GarageArea,1.000000,0.064245,0.000000
8,6,c_LotArea,1.000000,0.049686,0.000000
9,7,c_OpenPorchSF,1.000000,0.062331,0.000000


In [1137]:
vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,c_EnclosedPorch,1.000000,0.033443,0.000000
2,1,c_BsmtUnfSF,0.786901,0.037686,0.221445
3,2,c_2ndFlrSF,0.821109,0.031683,0.184744
5,3,c_WoodDeckSF,1.000000,0.043775,0.000000
6,4,c_MiscVal,1.000000,0.004830,0.000000
7,5,c_GarageArea,1.000000,0.064245,0.000000
8,6,c_LotArea,1.000000,0.049686,0.000000
9,7,c_OpenPorchSF,1.000000,0.062331,0.000000
10,8,c_MasVnrArea,1.000000,0.064245,0.000000


In [1138]:
columns=list(vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")["Variable"].values)

In [1139]:
columns

['c_EnclosedPorch',
 'c_BsmtUnfSF',
 'c_2ndFlrSF',
 'c_WoodDeckSF',
 'c_MiscVal',
 'c_GarageArea',
 'c_LotArea',
 'c_OpenPorchSF',
 'c_MasVnrArea']

## Arboles de decision

In [184]:
x_train=X_train[[x for x in X_train if x!="tgt_SalePrice"]]
y_train=X_train[[x for x in X_train if x=="tgt_SalePrice"]]

x_test=X_test[[x for x in X_test if x!="tgt_SalePrice"]]
y_test=X_test[[x for x in X_test if x=="tgt_SalePrice"]]

In [185]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [186]:

model = DecisionTreeRegressor()

model.fit(x_train, y_train)

importance = model.feature_importances_

In [187]:
importance

array([4.92438806e-02, 9.58682183e-03, 4.18852776e-01, 5.00538452e-04,
       5.29485972e-03, 9.48124015e-04, 1.48966330e-01, 1.05682330e-03,
       1.12741164e-02, 4.29033250e-04, 4.28385734e-02, 9.40703857e-05,
       0.00000000e+00, 5.67230856e-02, 1.10394580e-02, 2.42953505e-02,
       3.13411213e-03, 1.06733772e-02, 8.68939989e-02, 1.33345780e-03,
       3.03279890e-04, 1.09639811e-02, 3.59799885e-02, 2.17222018e-03,
       3.19062174e-06, 4.75159478e-03, 6.24921157e-04, 2.29956420e-05,
       2.36056971e-04, 3.71547645e-04, 5.75207890e-04, 8.36002321e-04,
       3.64893000e-04, 1.46858183e-03, 0.00000000e+00, 1.69614284e-03,
       9.82980615e-05, 6.23034852e-05, 2.74306159e-05, 1.11374031e-03,
       0.00000000e+00, 1.10665861e-03, 1.19070699e-03, 1.27643907e-04,
       8.78062111e-04, 2.58442378e-03, 4.97682931e-03, 1.72480765e-03,
       1.23105077e-03, 5.28987263e-04, 1.05597778e-03, 9.57619761e-03,
       2.39359899e-04, 2.21483405e-03, 1.49887891e-03, 4.24032481e-04,
      

In [188]:
pred_test=model.predict(x_test)
pred_train=model.predict(x_train)

In [189]:
r2_score(y_test,pred_test)

0.6042297583719747

In [190]:
r2_score(y_train,pred_train)

0.9999857648290367

In [191]:
importance

array([4.92438806e-02, 9.58682183e-03, 4.18852776e-01, 5.00538452e-04,
       5.29485972e-03, 9.48124015e-04, 1.48966330e-01, 1.05682330e-03,
       1.12741164e-02, 4.29033250e-04, 4.28385734e-02, 9.40703857e-05,
       0.00000000e+00, 5.67230856e-02, 1.10394580e-02, 2.42953505e-02,
       3.13411213e-03, 1.06733772e-02, 8.68939989e-02, 1.33345780e-03,
       3.03279890e-04, 1.09639811e-02, 3.59799885e-02, 2.17222018e-03,
       3.19062174e-06, 4.75159478e-03, 6.24921157e-04, 2.29956420e-05,
       2.36056971e-04, 3.71547645e-04, 5.75207890e-04, 8.36002321e-04,
       3.64893000e-04, 1.46858183e-03, 0.00000000e+00, 1.69614284e-03,
       9.82980615e-05, 6.23034852e-05, 2.74306159e-05, 1.11374031e-03,
       0.00000000e+00, 1.10665861e-03, 1.19070699e-03, 1.27643907e-04,
       8.78062111e-04, 2.58442378e-03, 4.97682931e-03, 1.72480765e-03,
       1.23105077e-03, 5.28987263e-04, 1.05597778e-03, 9.57619761e-03,
       2.39359899e-04, 2.21483405e-03, 1.49887891e-03, 4.24032481e-04,
      

In [193]:
scores=pd.DataFrame()
scores["gini_index"]=importance
scores["feature"]=x_train.columns

In [194]:
scores=scores.sort_values(by="gini_index",ascending=False)
scores.reset_index(drop=True,inplace=True)

In [195]:
feats=list(scores[:10]["feature"].values)

In [196]:
scores

,gini_index,feature
0,4.188528e-01,v_ExterQual
1,1.489663e-01,c_BsmtFinSF1
2,8.689400e-02,c_GarageArea
3,5.672309e-02,v_FullBath
4,4.924388e-02,c_LotArea
...,...,...
65,1.898050e-07,v_OverallCond_9
66,0.000000e+00,v_RoofMatl_otro
67,0.000000e+00,v_BsmtHalfBath
68,0.000000e+00,v_RoofStyle_otro
